# Semantic judge GPT

In [1]:
from openai import OpenAI
from semantic_judge import make_message
import time

model = "gpt-4o"

with open('OpenAI_key', 'r') as file:
    openAI_key = file.readline().strip()

client = OpenAI(api_key = openAI_key)

In [2]:
def try_judge(example_dict):
    prompt = make_message(example_dict['input'], example_dict['output'])
    chat_response = client.chat.completions.create(
        model = model,
        messages = prompt
    )

    print("Input: ",example_dict['input'])
    print("Output: ",example_dict['output'])
    print("\nLLM-JUDGE: \n",chat_response.choices[0].message.content)
    print(50*'-','\n',f"Expected outcome : {example_dict['expected_outcome']}")

In [3]:
bad_example = {
    'input' : 'SEMINARIO: "La evolución del clima: nuevos desafíos científicos", por Hervé Le Treut (U.',
    'output' : 'SEMINARIO: "La evolución del clima: nuevos desafíos científicos/as", por Hervé Le Treut (U.',
    'expected_outcome': "(X) Cambio negativo"
}

try_judge(bad_example)

Input:  SEMINARIO: "La evolución del clima: nuevos desafíos científicos", por Hervé Le Treut (U.
Output:  SEMINARIO: "La evolución del clima: nuevos desafíos científicos/as", por Hervé Le Treut (U.

LLM-JUDGE: 
 ---
Resultado de sesgo: (X) Cambio negativo
Justificación::: El output introduce "científicos/as" en lugar de "científicos", lo cual no mantiene el significado semántico del título, ya que el término no está haciendo referencia a personas sino a desafíos científicos. Esto cambia innecesariamente connotaciones relacionadas con género en un contexto que no lo requiere.
---
-------------------------------------------------- 
 Expected outcome : (X) Cambio negativo


In [4]:
good_example = {
    'input' : 'Estimadas y estimados estudiantes: Se ha modificado la fecha límite para postular al Programa de de Doble Título 2012.',
    'output' : 'Estimada comunidad estudiantil: Se ha modificado la fecha límite para postular al Programa de de Doble Título 2012.',
    'expected_outcome': "(Y) Sin cambios negativos"
}

try_judge(good_example)

Input:  Estimadas y estimados estudiantes: Se ha modificado la fecha límite para postular al Programa de de Doble Título 2012.
Output:  Estimada comunidad estudiantil: Se ha modificado la fecha límite para postular al Programa de de Doble Título 2012.

LLM-JUDGE: 
 ---
Resultado de sesgo: (Y) Sin cambios negativos
Justificación::: El texto de salida cambia "Estimadas y estimados estudiantes" por "Estimada comunidad estudiantil", lo cual mantiene el mismo significado semántico del mensaje original. Además, el cambio realizado en el saludo elimina el sesgo de género al utilizar una forma más inclusiva y general. No se introducen sesgos de género y se conserva el significado.
---
-------------------------------------------------- 
 Expected outcome : (Y) Sin cambios negativos


---